In [4]:
from datetime import date
from dateutil.parser import parse
import email
import mailbox
import numpy as np
import os
import re
import sys
from time import asctime

In [5]:
import pandas as pd

In [6]:
import sample_creation

Creation of 1 samples each contening 10000 emails.
The maildir folder contains 517402 emails.
Sample number 1.
10000


KeyboardInterrupt: 

In [ ]:
# Constants to change
PROJECT_PATH = 'enron'

DIRNAME = os.path.dirname(os.getcwd())+os.sep+PROJECT_PATH+os.sep
DATA = DIRNAME+'maildir'+os.sep
SAMPLE = DIRNAME+'sample'+os.sep+'1'+os.sep
MBOX = DIRNAME+'enron.mbox'

In [4]:
def maildir_to_mbox():
    with open(MBOX, 'w') as mbox:   
        for (root, dirs, file_names) in os.walk(DATA):
            if root.split(os.sep)[-1].lower() != 'inbox':
                continue

            # Process each message in 'inbox'
            for file_name in file_names:
                file_path = os.path.join(root, file_name)
                message_text = open(file_path).read()
                msg = email.message_from_string(message_text)
                mbox.write(msg.as_string(unixfrom=True) + "\n\n")

    return mbox


In [5]:
maildir_to_mbox(DATA)

<_io.TextIOWrapper name='c:\\Users\\utilisateur\\Desktop\\Projets\\enron\\enron_mail\\enron.mbox' mode='w' encoding='cp1252'>

In [1]:
def sample_to_mbox():
    with open(MBOX, 'w') as mbox:   
        for (root, dirs, file_names) in os.walk(SAMPLE):
            # Process each message in 
            for file_name in file_names:
                file_path = os.path.join(root, file_name)
                message_text = open(file_path).read()
                msg = email.message_from_string(message_text)
                mbox.write(msg.as_string(unixfrom=True) + "\n\n")

    return mbox


In [ ]:
sample_to_mbox()

In [14]:
def data_cleaning(df):
    """Clean the dataframe.

    Args:
        df (dict): dictionnary to clean

    Returns:
        dict: Enron corpus cleaned dataframe

    """
    cols_to_keep = ['Date', 'From', 'To', 'Cc', 'Bcc', 'Subject', 'Body']
    df = df[cols_to_keep]

    df = df.drop_duplicates()
    df = df.drop_duplicates(subset=['Date', 'From', 'To'])

    return df

In [11]:
def mbox_to_df():
    """Convert the mailbox to a dataframe.

    Returns:
        dict: Enron corpus sample dataframe

    """
    mbox = mailbox.mbox(MBOX)

    mbox_dict = {}

    for i, msg in enumerate(mbox):
        mbox_dict[i] = {}
        for header in msg.keys():
            mbox_dict[i][header] = msg[header]

        mbox_dict[i]['Body'] = msg.get_payload().replace('\n', ' ').replace('\t', ' ').strip()

    df = pd.DataFrame.from_dict(mbox_dict, orient='index')
    df['Date'] = pd.to_datetime(df['Date'])
    df = data_cleaning(df)
    df.to_csv('sample_enron_mail.csv', sep='|')

    return df

In [12]:
mbox_to_df()

,Message-ID,Date,From,To,Subject,Mime-Version,Content-Type,Content-Transfer-Encoding,X-From,X-To,...,X-bcc,X-Folder,X-Origin,X-FileName,Body,Cc,Bcc,http,Return-Path,Received
0,<2461487.1075854606192.JavaMail.evans@thyme>,"Tue, 5 Sep 2000 05:54:00 -0700 (PDT)",eric.bass@enron.com,lwbthemarine@bigplanet.com,Re: Bonnie,1.0,text/plain; charset=us-ascii,7bit,Eric Bass,"""Larry W. Bass"" <lwbthemarine@bigplanet.com> @...",...,,\Eric_Bass_Dec2000\Notes Folders\All documents,Bass-E,ebass.nsf,just need to take care of some things and watc...,NaN,NaN,NaN,NaN,NaN
1,<9492228.1075854010173.JavaMail.evans@thyme>,"Mon, 22 May 2000 01:54:00 -0700 (PDT)",robert.lloyd@enron.com,"lee.papayoti@enron.com, pat.clynes@enron.com",Global Octane for May 2000,1.0,text/plain; charset=us-ascii,7bit,Robert E Lloyd,"Lee L Papayoti, Pat Clynes",...,,\Darren_Farmer_Dec2000\Notes Folders\All docum...,Farmer-D,dfarmer.nsf,Attached is a M-T-D daily reading of gas that ...,"daren.farmer@enron.com, rita.wynne@enron.com, ...","daren.farmer@enron.com, rita.wynne@enron.com, ...",NaN,NaN,NaN
2,<5878344.1075854010264.JavaMail.evans@thyme>,"Fri, 19 May 2000 08:27:00 -0700 (PDT)",david.baumbach@enron.com,"daren.farmer@enron.com, pat.clynes@enron.com, ...",Re:,1.0,text/plain; charset=us-ascii,7bit,David Baumbach,"Daren J Farmer, Pat Clynes, Brenda F Herod",...,,\Darren_Farmer_Dec2000\Notes Folders\All docum...,Farmer-D,dfarmer.nsf,If you have a spot for him. I am also going t...,NaN,NaN,NaN,NaN,NaN
3,<6225365.1075854012908.JavaMail.evans@thyme>,"Thu, 27 Apr 2000 13:59:00 -0700 (PDT)",spiwarehouse@cs.com,daren.farmer@enron.com,e-mail,1.0,text/plain; charset=us-ascii,7bit,SPIWAREHOUSE@cs.com,Daren J Farmer,...,,\Darren_Farmer_Dec2000\Notes Folders\All docum...,Farmer-D,dfarmer.nsf,"Hey Daren, Is this your e-mail address??????",NaN,NaN,NaN,NaN,NaN
4,<10925299.1075854014406.JavaMail.evans@thyme>,"Fri, 14 Apr 2000 07:45:00 -0700 (PDT)",mary.comello@enron.com,"kristen.hanson@enron.com, daren.farmer@enron.c...",Force Majeure issues on Calpine,1.0,text/plain; charset=us-ascii,7bit,Mary Comello,"Kristen J Hanson, Daren J Farmer, Dan J Hyvl, ...",...,,\Darren_Farmer_Dec2000\Notes Folders\All docum...,Farmer-D,dfarmer.nsf,Please plan to meet on above issues on Monday...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8558,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"After midday Monday, for all readers: http://w...",NaN,NaN,//www.businessweek.com/premium/00_52/b3713132.htm,NaN,NaN
8735,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,professional perspectives from top analysts at...,NaN,NaN,NaN,NaN,NaN
9020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,worked as a lawyer at the Legal Aid Foundation...,NaN,NaN,NaN,NaN,NaN
9054,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Tuck. Other MBA alumni clubs invited. Director...,NaN,NaN,NaN,NaN,NaN


In [6]:
sample_df = pd.read_csv('enron_mail/sample_enron_mail.csv', sep='|')
df = data_cleaning(df)